<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Multi-Variable Logistic Regression and Classification Matrix

_Authors: Sam Stack(DC)_


**Exercise Objectives**
- Hand on experience using Multi-Variable Logistic Regression
- Review and Exploration of the Classification Matrix and its evaluation Metrics
- Introduction to One vs. One and One vs. Rest Classifiers.

**Lets get some data.**
One of the most popular classification datasets for Machine learning is the Iris Dataset, which can be loaded directly from `sklearn.datasets`
- Sklearn datasets are imported as dictionaries and use keys to access specific aspects.
    - `iris.data` : actual matrix of observations
    - `iris.target` : target column for classification
    - `iris.feature_names` :  column names

In [36]:
import seaborn as sns
import pandas as pd
from sklearn import datasets

iris = datasets.load_iris()

X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = iris.target

In [37]:
# Examine the data
X.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [4]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

**Break down of classes**  
0 : Setosa  
1 : Versicolour  
2 : Virginica  

----

**Modelling**
This data is extreamly neat and tidy so no cleaning necessary and we can get right into modelling.

In [38]:
# model the data, use a cross validation technique as well
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

lr = LogisticRegression()

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=1237)

lr.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [21]:
# model.predict
y_pred = lr.predict(X_test)

In [39]:
lr.score(X_test, y_test)

0.9333333333333333

In [45]:
from sklearn.model_selection import KFold

X_2 = iris.data

kf = KFold(n_splits=2)
kf.get_n_splits(X_2)
# KFold(n_splits=2, random_state=None, shuffle=False)

for train_index, test_index in kf.split(X_2):
    X_train, X_test = X_2[train_index], X_2[test_index]
    y_train, y_test = y[train_index], y[test_index]
    lr = LogisticRegression()
    lr.fit(X_train, y_train)
    print(lr.score(X_test, y_test))

0.28
0.3333333333333333


In [22]:
# evaluated model preformance with a confusion matrix.
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[14,  0,  0],
       [ 0, 14,  3],
       [ 0,  0, 14]])

With a multivariable confusion matrix, some of our labellings (True Pos., True Neg., False Pos., False Neg.) get a little warped.  We are no longer predicting one class from a null class we are classifiying into 3 distinguished classes.  

The **True** diagonal stays the same as these are properly classified observations.  


|     | Class 0 | Class 1  | Class 2 |
| --- | ------- |:--------:| -------:|
| **Pred Class 0**  | 15      | 0        | 0       |
| **Pred Class 1**    | 0       | 11       |   0     |
| **Pred Class 2**    | 0       | 1        |    11   |


It is better to stick with True and False labels with multi-class to avoid ...[_Confusion_](https://www.youtube.com/watch?v=bcYppAs6ZdI)

If you need to reffer to a False Positive or True Negative it is better to first select a specific class, such as `Class 2 ` and refer to classification or misclassification relative to said choosen class instead of the set of all classes as a whole. 

Example:
    _True Negatives relative to Class 2 are True Positives for Class 0 and Class 1._

Speaking of our Classes?  How are probabilities calculated with multi class?
- Are they Probability of `Class 0` vs. `Not Class 0`?
- Or Probability of `Class 0` vs. `Class 1` vs. `Class 2` ?

In [23]:
# use predict_proba to find out.
lr.predict_proba(X_test)

array([[8.21307146e-01, 1.78480491e-01, 2.12362972e-04],
       [1.74527908e-03, 3.09132378e-01, 6.89122343e-01],
       [1.60207636e-02, 3.33156722e-01, 6.50822515e-01],
       [5.76650143e-03, 3.06536793e-01, 6.87696706e-01],
       [4.85635820e-03, 5.65383868e-01, 4.29759774e-01],
       [3.51728976e-03, 3.79663301e-01, 6.16819409e-01],
       [3.55370736e-03, 4.70269540e-01, 5.26176752e-01],
       [9.20053884e-01, 7.98824626e-02, 6.36529649e-05],
       [1.94631071e-01, 6.22072106e-01, 1.83296823e-01],
       [5.65848903e-04, 2.62285795e-01, 7.37148356e-01],
       [8.46017539e-01, 1.53918340e-01, 6.41207822e-05],
       [8.92565978e-04, 1.91603045e-01, 8.07504389e-01],
       [2.10622098e-03, 1.24565646e-01, 8.73328133e-01],
       [4.73477123e-02, 7.37313802e-01, 2.15338485e-01],
       [4.93020249e-02, 6.18698090e-01, 3.31999885e-01],
       [8.06795561e-01, 1.92865286e-01, 3.39153002e-04],
       [2.78233998e-02, 5.77606186e-01, 3.94570414e-01],
       [7.09997326e-03, 2.95948

In [24]:
for a,b,c in lr.predict_proba(X_test):
    print(sum([a,b,c]))

1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
0.9999999999999999
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
0.9999999999999998
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
1.0
1.0
1.0
1.0
0.9999999999999999
1.0
0.9999999999999999
1.0
0.9999999999999998
1.0
1.0
1.0
1.0
1.0


Looks like our probabilities of each class all add up to 1, so it is like `Class 0` vs. `Class 1` vs. `Class 2`.

What if we wanted to create a logistic regression that has `Class 0` vs. `Class 1` & `Class 2` or just `Class 0` vs. `Class 2`?  We will cover that in a bit, but first more evaluation metrics.

---

**Classification Reports/Matrix**

Classification reports are another means of evauliation classification models and return a few metrics that are based on True Positives, False Positives and False Negatives.  

In [27]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        14
          1       1.00      0.82      0.90        17
          2       0.82      1.00      0.90        14

avg / total       0.95      0.93      0.93        45



**Precision**  
- "How many of the items selected are relevant."
- Of the items placed into a class, how many of the are True Positives.


$$\frac{True Positives}{True Positives + False Positives}$$

**Recall**  
- "How many of the relevant items are selected."
- Of the items that were suppose to be placed into a class, how many did we accurately place.


$$\frac{True Positives}{True Positives + False Negatives}$$

**F1-Score**

F1 exists on a range of 0 - 1 where 0 is just aweful and 1 is perfection.
F1 is considered a harmonic mean as it averages Precision and Recall.  With classification models you often times have to chooise what kind of error you are willing to increase in order to reduce the other and thus you may want to optimize Precision or Recall accordingly.  If you are uncertain which you should optimize, F1 score may be the metric of choice.

$$2*\frac{precision * recall}{precision + recall}$$

**Support**
Number of true observations in given class.  The count of possible true observations.  

---

## Intro to Ensembling

Earlier we talked about building models relative to class combinations.  Distinguishing One class from all other classes or just One specific class from another specific class.  These goals are possible with Logistic Regression.

Up until this point we have used one model, but there are also Machine Learning methods that involve combining several models to come to a more refined conclusion, commonly reffered to as Ensemble Methods.

### One Vs. Rest Classification.

One vs. Rest Classification is a method that builds an individual model for each class to try to distingush said specific class from the rest of the classes.  Since we are only focusing on one class, `Class 1` these classfiers will group `Class2`, `Class3`, `Class4` into a single class of `Not Class 1`.  Same all the way through for the rest of the classes.

1 - Class1 vs. Class2, Class3, Class4  
2 - Class2 vs. Class1, Class3, Class4  
3 - Class3 vs. Class1, Class2, Class4   
4 - Class4 vs. Class1, Class2, Class3  

### One Vs. One Classification.

We train a model for every set of classes.  As more classes are added this becomes more computationally expense.  

1 - Class1 vs. Class2  
2 - Class1 vs. Class3  
3 - Class1 vs. Class4  
4 - Class2 vs. Class3  
5 - Class2 vs. Class4  
6 - Class3 vs. Class4  


#### One Vs. Rest Classifier

In [28]:
# import ensemble method
from sklearn.multiclass import OneVsRestClassifier 
# instantiate choose model
LR = LogisticRegression()
# place the model in the ensembler
OnevsRest = OneVsRestClassifier(LR)
# use the ensemble like a normal sklearn model.
OnevsRest.fit(X_train, y_train)


# you can use the train test split you created earlier or do a new TTS.

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          n_jobs=1)

In [32]:
# use the .predict and confusion matrix the same way
y_pred = OnevsRest.predict(X_test)

confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        14
          1       1.00      0.82      0.90        17
          2       0.82      1.00      0.90        14

avg / total       0.95      0.93      0.93        45



#### One Vs. One Classifier

In [30]:
from sklearn.multiclass import OneVsOneClassifier
LR = LogisticRegression()
# OvO works the same as OvR
OneVsOne = OneVsOneClassifier(LR)
OneVsOne.fit(X_train, y_train)

OneVsOneClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
          n_jobs=1)

In [33]:
# Make prediction and evaluate confusion matrix.
y_pred = OneVsOne.predict(X_test)

confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        14
          1       1.00      0.94      0.97        17
          2       0.93      1.00      0.97        14

avg / total       0.98      0.98      0.98        45



One Vs. One/Rest Classifiers are not restricted to fitting using Logistic Regression.  With SKLearn, any type of Classification model can be placed into the One Vs X classification ensemble.

In [35]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=2)

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


lr = LogisticRegression()
lr.fit(X_train, y_train)
print(lr.score(X_test, y_test))

KeyError: '[ 75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92\n  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110\n 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128\n 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146\n 147 148 149] not in index'